In [1]:
!pip install ruptures

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /burg/opt/anaconda3-2023.09/lib/python3.11/site-packages/singularity_hpc-0.1.28-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 64.2 MB/s eta 0:00:000m eta 0:00:010:01:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import pickle
from typing import Dict,List,Tuple
from functools import partial
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from scipy.stats import gaussian_kde
from matplotlib.colors import LinearSegmentedColormap

import os
import glob
import pickle
from datetime import datetime
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pandas as pd
import pandas.api.types as ptypes
import matplotlib.pyplot as plt
from sklearn.preprocessing import QuantileTransformer, LabelEncoder, StandardScaler, MinMaxScaler

/burg/opt/anaconda3-2023.09/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
data_path = '/burg/glab/users/al4385/data/TFT_30/BDT_50_90.pickle'
file_path = '/burg/glab/users/al4385/predictions/TFT_30_0429/pred_BDT_50_90.pkl'
coord_path = '/burg/glab/users/al4385/data/coordinates/BDT_50_90.parquet'


#file_path = "predictions/predictions_BDT_boreal_jan6_timesplit.pkl" #validation outputs 
with open(file_path, 'rb') as file:
    # Load the dictionary from the pickle file
    preds = pickle.load(file)

#data_path = '../data/TFT_365test_03292024/BDT_50_90_test_NA.pickle'
with open(data_path,'rb') as fp:
        data = pickle.load(fp)

coords=pd.read_parquet(coord_path)

In [ ]:
df = pd.DataFrame({
    'Index': data['data_sets']['test']['id'].flatten(),
    'Flattened_Values': data['data_sets']['test']['target'].flatten(),
})

# Separate the channels into three separate arrays
channel1 = preds['predicted_quantiles'][:, :, 0]
channel2 = preds['predicted_quantiles'][:, :, 1]
channel3 = preds['predicted_quantiles'][:, :, 2]

#seperate weights into different channels 
df['tmin_weight_f'] = preds['future_selection_weights'][:, :, 0].flatten()
df['tmax_weight_f'] = preds['future_selection_weights'][:, :, 1].flatten()
df['rad_weight_f'] = preds['future_selection_weights'][:, :, 2].flatten()
df['precip_weight_f'] = preds['future_selection_weights'][:, :, 3].flatten()
df['photoperiod_weight_f'] = preds['future_selection_weights'][:, :, 4].flatten()
df['SM_weight_f'] = preds['future_selection_weights'][:, :, 5].flatten()

#df['tmin_weight_h'] = preds['historical_selection_weights'][:, :, 1].flatten()
#df['tmax_weight_h'] = preds['historical_selection_weights'][:, :, 2].flatten()
#df['rad_weight_h'] = preds['historical_selection_weights'][:, :, 3].flatten()
#df['precip_weight_h'] = preds['historical_selection_weights'][:, :, 4].flatten()
#df['photoperiod_weight_h'] = preds['historical_selection_weights'][:, :, 5].flatten()


df['pred_01']= channel1.flatten()
df['pred_05']= channel2.flatten()
df['pred_09']= channel3.flatten()

df[['location_id', 'time_id']] = df['Index'].str.split('_', n=1, expand=True)

df['time_id'] = pd.to_datetime(df['time_id'])
df['doy']=df['time_id'].dt.dayofyear
df['location_id'] = df['location_id'].astype(int)
df = df.sort_values(by=['location_id','time_id'])

df['year']= df['time_id'].dt.year
df['month']= df['time_id'].dt.month
df['day']= df['time_id'].dt.day
df = df.drop(columns=['Index'])

In [ ]:
data_df = coords#['test']
data_df = data_df.rename(columns={ 'location': 'location_id'})
data_df = data_df.drop_duplicates()

both = pd.merge(df, data_df, on='location_id', how='left')
df_MAM=both